In [4]:
include("OperatorAlgebra.jl")
include("Krylov.jl")

false

In [6]:
Id=unit(MajoranaOperator)
Id*maj(4,5)

χ4 χ5

Building Hubbard model with different operators to compare the ground state 

In [8]:
function orb(i::Int, s::Int)
    return 2i + s
end
function FHubbard(L::Int, t::Number, U::Number)
    T = UInt64   # UInt8 for L≤4, UInt16 for 5..7, else wider
    H = zero(FermionOperator{T})

    # Hopping: -t * Σ_{i,s} ( c†_{i,s} c_{i+1,s} + c†_{i+1,s} c_{i,s} ), Periodic
    for i in 0:L-1
        ip1 = (i + 1) % L
        for s in (0, 1)
            j = orb(i, s)
            jp = orb(ip1, s)
            H -= t * (cdag(T, j) * c(T, jp) + cdag(T, jp) * c(T, j))
        end
    end

    # On-site interaction: U * Σ_i n_{i,↑} n_{i,↓}
    for i in 0:L-1
        jdn = orb(i, 0)
        jup = orb(i, 1)

        H += U * (n(T, jdn) * n(T, jup))
    end
    return H
end


FHubbard (generic function with 1 method)

In [ ]:
mc(i::Int, s::Int)  = (maj(4*i + 2*s) + 1im*maj(4*i + 2*s + 1)) / 2
mcd(i::Int, s::Int) = (maj(4*i + 2*s) - 1im*maj(4*i + 2*s + 1)) / 2
mn(i::Int, s::Int)  = mcd(i,s) * mc(i,s)

function MHubbard(L::Int, t::Number, U::Number)
    H = zero(MajoranaOperator{UInt64})
    for i in 0:L-1
        ip1 = (i + 1) % L
        for s in (0, 1)
            H -= t * (mcd(i, s) * mc(ip1, s) + mcd(ip1, s) * mc(i, s))
        end
    end
    for i in 0:L-1
        H += U * (mn(i, 0) * mn(i, 1))
    end
    return H
end


In [ ]:

@inline function JW_String(j::Int)::PauliOperator{UInt64}
    j <= 0 && return unit(PauliOperator{UInt64})
    j > 63 && error("PKey is 64-bit; j must be ≤ 63 (got j=$j)")
    zmask = (UInt64(1) << UInt64(j)) - UInt64(1)   # bits 0..j-1 set
    return PauliOperator{UInt64}(Dict(PKey(UInt64(0), zmask) => 1.0 + 0im))
end

# Raising/lowering on a single site
@inline σplus(j::Int)  = (px(j) + (0+1im)*py(j)) / 2     # (X + iY)/2
@inline σminus(j::Int) = (px(j) - (0+1im)*py(j)) / 2     # (X - iY)/2

# --------- Fermionic operators (Jordan–Wigner) ----------
# Convention: |0> = empty, |1> = occupied  ⇒  n = (I - Z)/2
c(j::Int) = JW_String(j) * σplus(j)     # c_j
cd(j::Int)     = JW_String(j) * σminus(j)    # c†_j

# Number operator (two equivalent ways)
n_op(j::Int)  = cd(j) * c(j)              # c†_j c_j
n_op2(j::Int) = (unit(PauliOperator{UInt64}) - pz(j)) / 2      # (I - Z_j)/2





function PHubbard(L::Int, t::Number, U::Number)
    H = zero(PauliOperator{UInt64})  

    # Hopping: -t * Σ_{i,s} ( c†_{i,s} c_{i+1,s} + c†_{i+1,s} c_{i,s} )
    for i in 0:L-1
        ip1 = (i + 1) % L                   # periodic neighbor
        for s in (0, 1)
            H -= t * (cd(2*i + s) * c(2*ip1+ s) + cd(2*ip1 + s) * c(2*i+s))
        end
    end

    # On-site interaction: U * Σ_i n_{i,↑} n_{i,↓}
    for i in 0:L-1
        H += U * (n_op(2*i+ 0) * n_op2(2i+ 1))
    end

    return H
end



In [9]:
Hamiltonian=FHubbard(6,1,1)
rho = qboot(Hamiltonian, show_progress = true, max_power = 20, closure_depth = 24, 
max_basis_size = 40000000, enable_reorth=false,reorth_first=0)

println("Ground state energy: ", inner(Hamiltonian, rho))


>>> Building H-generated monomial space
closure depth 1: frontier=1, total=1
closure depth 2: frontier=30, total=31
closure depth 3: frontier=447, total=478
closure depth 4: frontier=4092, total=4570
closure depth 5: frontier=24627, total=29197
closure depth 6: frontier=97518, total=126715
closure depth 7: frontier=239011, total=365726
closure depth 8: frontier=309942, total=675668
closure depth 9: frontier=158802, total=834470
closure depth 10: frontier=18574, total=853044
closure depth 11: frontier=720, total=853764
closure depth 12: frontier=12, total=853776
H-generated space: 853776 monomials (depth=12)
Frozen basis size: 853776 monomials
    k   terms   weight      norm
 0:      1    1.000 1.000e+00
  [apply_action] : 0.0013270378112792969 seconds
alpha= 1.500000e+00  norm= 2.669270e+00
 1:     31    4.526 2.669e+00
  [apply_action] : 0.0025949478149414062 seconds
alpha= 1.578947e+00  norm= 3.450832e+00
 2:    478   21.781 3.451e+00
  [apply_action] : 0.0015261173248291016 seconds